## 参考文献汇总
- Reference 1: Kaggle Kernel [Code sharing, 13th place, no external data](https://www.kaggle.com/c/rossmann-store-sales/discussion/17979)
- Reference 2: Kaggle Kernel-[A Journey through Rossmann Stores](https://www.kaggle.com/omarelgabry/rossmann-store-sales/a-journey-through-rossmann-stores)
- Reference 3: Kaggle Kernel-[Predict_sales_with_pandas](https://www.kaggle.com/zygmunt/rossmann-store-sales/predict-sales-with-pandas-py)
- Reference 4:[XGBoost:Notes on Parameter Tuning](http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html)
- Reference 5:[XGBoost Parameters](http://xgboost.readthedocs.io/en/latest/parameter.html)
- Reference 6:[Complete Guide to Parameter Tuning in XGBoost (with codes in Python)](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)
- Reference 7:[XGBoost Feature Importance](https://www.kaggle.com/cast42/rossmann-store-sales/xgboost-in-python-with-rmspe-v2)


## 程序使用说明
1. 运行程序前，从[Kaggle](https://www.kaggle.com/c/rossmann-store-sales/data)官网下载数据store.csv.zip、train.csv.zip和test.csv.zip，将解压后的store.csv、train.csv、test.csv放在与本文档同目录下的data文件夹中
2. "RossmanSales_Part_01_DataPreprocessing.ipynb"会生成后续数据可视化、模型训练所需要的数据
4. "RossmanSales_Part_02_VisualExploration.ipynb"为数据可视化，需使用Part 1生成的数据
5. “RossmanSales_Part_03_BenchmarkModel.ipynb”生成基准模型，需使用Part 1生成的数据
6. “RossmanSales_Part_04_XGBoost_HyperPara_Optimization.ipynb”使用XGBoost方法训练模型，并进行超参数优化；需使用Part 1生成的数据；
7. 文件夹说明：
    - data： 保存store.csv、train.csv、test.csv
    - data_inter_transfer：保存"RossmanSales_Part_01_DataPreprocessing.ipynb"生成的数据
    - output_csv：保存用来提交到Kaggle的结果文件
    - model_saved：保存生成的模型

## 准备工作
### 0.1 调入库函数

In [2]:
#基本计算类
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

#可视化
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display 

#xgboost库函数
import xgboost as xgb


#时间类
import time
import datetime
from isoweek import Week

#文件类
import os

#其他
import itertools
import operator

#基本设定

%matplotlib inline
sns.set_style('whitegrid')

### 0.2 定义评估指标
本项目采用Kaggle比赛的评估指标：RMSPE（误差百分比的均方差），可表示为
$$
RMSPE= \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}
$$
其中，任何当天销售额为0的数据在评估时将被忽略； $y_i$ 表示某药店在某天的实际销售额，而$\hat{y_i}$ 表示该药店在对应这一天的预测销售额。

### 0.3 Setting seed

In [3]:
seed_1=42

## 1. 读入数据，基本数据处理 & 预览数据

### 1.1 读入数据
#### 1.1.0 读入数据

In [4]:
train_raw_df=pd.read_csv('data/train.csv',low_memory=False)
test_raw_df=pd.read_csv('data/test.csv',low_memory=False)
store_raw_df=pd.read_csv('data/store.csv',low_memory=False)

#### 1.1.1 数据探索

In [5]:
### 对数据进行预览 1
print('--------train.csv---------')
display(train_raw_df.head(n=2))
print('--------test.csv---------')
display(test_raw_df.head(n=2))
print('--------store.csv---------')
display(store_raw_df.head(n=2))

--------train.csv---------


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


--------test.csv---------


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0


--------store.csv---------


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [6]:
#对数据进行预览 2
print('--------train.csv---------')
train_raw_df.info()
print('--------test.csv---------')
test_raw_df.info()
print('--------store.csv---------')
store_raw_df.info()

--------train.csv---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB
--------test.csv---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
Id               41088 non-null int64
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Date             41088 non-null object
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null int64
dtypes

In [7]:
print('--------store.csv---------')
store_raw_df.describe()

--------store.csv---------


/Users/pengjuzhao/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,NaN,NaN,NaN,0.000000,NaN,NaN
50%,558.00000,NaN,NaN,NaN,1.000000,NaN,NaN
75%,836.50000,NaN,NaN,NaN,1.000000,NaN,NaN
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [8]:
#对数据进行预览 3
print('--------train.csv---------')
train_raw_df.describe()

--------train.csv---------


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


In [9]:
#对数据进行预览 3
print('--------test.csv---------')
test_raw_df.describe()

--------test.csv---------


,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,NaN,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,NaN,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,NaN,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


#### 1.1.2 train.csv数据探索

In [10]:
print'max store id:',train_raw_df.Store.unique().max()
print'min store id:',train_raw_df.Store.unique().min()

max store id: 1115
min store id: 1


In [11]:
print'max DayOfWeek:',train_raw_df.DayOfWeek.unique().max()
print'min DayOfWeek:',train_raw_df.DayOfWeek.unique().min()

max DayOfWeek: 7
min DayOfWeek: 1


In [12]:
print'ealiest date:',train_raw_df.Date.unique().min()
print'latest date:',train_raw_df.Date.unique().max()

ealiest date: 2013-01-01
latest date: 2015-07-31


In [13]:
print 'Open Unique value:', train_raw_df.Open.unique()

Open Unique value: [1 0]


In [14]:
print 'Promo Unique value:', train_raw_df.Promo.unique()

Promo Unique value: [1 0]


In [15]:
print 'StateHoliday Unique value:', train_raw_df.StateHoliday.unique()

StateHoliday Unique value: ['0' 'a' 'b' 'c']


In [16]:
print 'SchoolHoliday Unique value:', train_raw_df.SchoolHoliday.unique()

SchoolHoliday Unique value: [1 0]


#### 1.1.3 test.csv数据探索

In [17]:
print'Unique DayOfWeek value:',np.sort(test_raw_df.DayOfWeek.unique())

Unique DayOfWeek value: [1 2 3 4 5 6 7]


In [18]:
print 'Open Unique value:', test_raw_df.Open.unique()

Open Unique value: [  1.  nan   0.]


In [19]:
print 'Promo Unique value:', test_raw_df.Promo.unique()

Promo Unique value: [1 0]


In [20]:
print 'SchoolHoliday Unique value:', test_raw_df.SchoolHoliday.unique()

SchoolHoliday Unique value: [0 1]


In [22]:
print 'Date Unique value:', test_raw_df.Date.unique()
print len(test_raw_df.Date.unique())

Date Unique value: ['2015-09-17' '2015-09-16' '2015-09-15' '2015-09-14' '2015-09-13'
 '2015-09-12' '2015-09-11' '2015-09-10' '2015-09-09' '2015-09-08'
 '2015-09-07' '2015-09-06' '2015-09-05' '2015-09-04' '2015-09-03'
 '2015-09-02' '2015-09-01' '2015-08-31' '2015-08-30' '2015-08-29'
 '2015-08-28' '2015-08-27' '2015-08-26' '2015-08-25' '2015-08-24'
 '2015-08-23' '2015-08-22' '2015-08-21' '2015-08-20' '2015-08-19'
 '2015-08-18' '2015-08-17' '2015-08-16' '2015-08-15' '2015-08-14'
 '2015-08-13' '2015-08-12' '2015-08-11' '2015-08-10' '2015-08-09'
 '2015-08-08' '2015-08-07' '2015-08-06' '2015-08-05' '2015-08-04'
 '2015-08-03' '2015-08-02' '2015-08-01']
48


#### 1.1.4 store.csv数据探索

In [16]:
print 'StoreType:',store_raw_df.StoreType.unique()

StoreType: ['c' 'a' 'd' 'b']


In [17]:
print 'Assortment:',store_raw_df.Assortment.unique()

Assortment: ['a' 'c' 'b']


In [18]:
print 'Promo2:',store_raw_df.Promo2.unique()

Promo2: [0 1]


In [19]:
print 'PromoInterval:',store_raw_df.PromoInterval.unique()

PromoInterval: [nan 'Jan,Apr,Jul,Oct' 'Feb,May,Aug,Nov' 'Mar,Jun,Sept,Dec']


### 1.2  train_raw_df，test_raw_df数据处理


#### 1.2.1 train_raw_df，test_raw_df的时间数据处理
- Year,Month,Day
- DaysThisYear

In [52]:
#将str格式Date转换为datetime64格式Date
train_raw_df['Date_time']=pd.to_datetime(train_raw_df['Date'])
test_raw_df['Date_time']=pd.to_datetime(test_raw_df['Date'])
#提取int格式的Year，Month，Day
var_name='Date_time'
train_raw_df['Year']=( pd.Index(train_raw_df[var_name]).year).astype(np.int64)
train_raw_df['Month']=( pd.Index(train_raw_df[var_name]).month).astype(np.int64)
train_raw_df['Day']=( pd.Index(train_raw_df[var_name]).day).astype(np.int64)
train_raw_df['Year-Month']=train_raw_df['Date'].apply(lambda x:(str(x)[:7]))

test_raw_df['Year']=( pd.Index(test_raw_df[var_name]).year).astype(np.int64)
test_raw_df['Month']=( pd.Index(test_raw_df[var_name]).month).astype(np.int64)
test_raw_df['Day']=( pd.Index(test_raw_df[var_name]).day).astype(np.int64)
test_raw_df['Year-Month']=test_raw_df['Date'].apply(lambda x:(str(x)[:7]))


#将Month转换为string
month2str={
    1:'Jan',2:'Feb',3:'Mar',4:'Apr',
    5:'May',6:'Jun',7:'Jul',8:'Aug',
    9:'Sept',10:'Oct',11:'Nov',12:'Dec'
}
train_raw_df['Month-Str']=train_raw_df['Month'].map(month2str)
train_raw_df['DayOfYear']=train_raw_df.Date_time.dt.dayofyear


test_raw_df['Month-Str']=test_raw_df['Month'].map(month2str)
test_raw_df['DayOfYear']=test_raw_df.Date_time.dt.dayofyear

#### 1.2.2 处理test_raw_df Open特征的缺失值
- 由评估指标 RMSPE（误差百分比的均方差）的定义可知，RMSPE仅与非零Sales有关；同时考虑到所有Open=0时，Sales也为0，那么将Open的缺失值全部设置为1，将不会引入新的误差：
    - 如果缺失值对应的实际Sales为零，那么即使预测Sales非零，也不会被计算入RMSPE；
    - 如果缺失值对应的实际Sales非零，那么预测Sales将会被计算入RMSPE；

In [53]:
test_raw_df.Open.fillna(1,inplace=True)

#### 1.2.3 将train_raw_df中的Sales转换为SalesLog

In [54]:
# train_raw_df['SalesLog']=np.log(train_raw_df['Sales'])
train_raw_df['SalesLog']=np.log(train_raw_df['Sales'])

#### 1.2.4 将train_raw_df，test_raw_df的StateHoliday转换为分类数据

In [55]:
StateHoliday_dict={
    '0':0, 'a':1, 'b':2, 'c':3
}
train_raw_df['StateHoliday_cat']=train_raw_df['StateHoliday'].map(StateHoliday_dict)
test_raw_df['StateHoliday_cat']=test_raw_df['StateHoliday'].map(StateHoliday_dict)

#### 1.2.5 train_raw_df异常值处理
- 在模型训练过程中，使用高质量的训练数据是非常重要的，其中一个重要方面是要剔除异常值
- 使用MAD（median absolute deviation）方法来标记异常值

In [56]:
def mad_based_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh


for i in train_raw_df['Store'].unique():    
    train_raw_df.loc[(train_raw_df['Store'] == i) & (train_raw_df['Open'] == 1), 'Outlier_3'] = \
        mad_based_outlier(train_raw_df.loc[ (train_raw_df['Store'] == i) & (train_raw_df['Open'] == 1)]['Sales'], 3)

### 1.3 store_raw_df数据处理
#### 1.3.1 store_raw_df的Competition数据处理
- 目的：
    - store_raw_df中添加特征InCompetition，CompetitionSinceDate
    - 当CompetitionDistance为缺省值时，CompetitionOpenSinceMonth，CompetitionOpenSinceYear亦同时为缺省值，在这种情况下，假定该药店开始竞争的时间为train.csv统计时间结束时（取为2016-01-01），而竞争距离为CompetitionDistance的中位数；
    - 当CompetitionDistance不缺省值，而CompetitionOpenSinceMonth，CompetitionOpenSinceYear为缺省值时，假定Competition开始时间太早而无法统计到，将其统一设置为CompetitionOpenSinceMonth/Year的最早值1961-01-01。

- 通过验证可知，CompetitionOpenSinceYear与CompetitionOpenSinceMonth同时为NaN or 非NaN
- 如果CompetitionDistance 或者CompetitionOpenSinceMonth/Year其中一个不为NaN，则认为该store的InCompetition为True;
- 对于CompetitionDistance不为NaN，而CompetitionOpenSinceMonth为NaN的store，**假设**认为Competition开始时间太早而无法统计到，将其统一设置为CompetitionOpenSinceMonth/Year的最早值1961-01-01
- 对于CompetitionDistance为NaN的store，**假设** CompetitionOpenSinceMonth /Year 设定为一个在test样本之后的日期2016-01-01,CompetitionDistance的缺失值设为中位数。

In [57]:
store_raw_df[store_raw_df.CompetitionDistance.isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [58]:
print("count of CompetitionOpenSinceYear!=CompetitionOpenSinceMonth is {}".format(
        np.sum(store_raw_df.CompetitionOpenSinceYear.isnull()!=store_raw_df.CompetitionOpenSinceMonth.isnull()
              )))

#如果CompetitionDistance 或者CompetitionOpenSinceMonth/Year其中一个不为NaN，则认为该store的InCompetition为True
store_raw_df['InCompetition']=0
store_raw_df['InCompetition']=np.logical_or(
        store_raw_df.CompetitionDistance.notnull(),
        store_raw_df.CompetitionOpenSinceMonth.notnull())

# store_raw_df中CompetitionDistance为NaN的设为CompetitionDistance的中位数
store_raw_df['CompetitionDistance'].fillna(
    np.median(store_raw_df['CompetitionDistance'][store_raw_df['CompetitionDistance'].notnull()]),
    inplace=True)
#将CompetitionDistance不为NaN，而CompetitionOpenSinceMonth/Year为NaN设置为1961-01-01
store_raw_df['CompetitionOpenSinceYear'].fillna(1961,inplace=True)
store_raw_df['CompetitionOpenSinceMonth'].fillna(1,inplace=True)
# 将CompetitionDistance为NaN的CompetitionOpenSinceMonth/Year设置为2016-01-01
store_raw_df.loc[store_raw_df['InCompetition']==0,'CompetitionOpenSinceYear']=2016
# 将CompetitionOpenSinceMonth/Year格式改为np.int64
store_raw_df['CompetitionOpenSinceMonth']=store_raw_df['CompetitionOpenSinceMonth'].astype(np.int64)
store_raw_df['CompetitionOpenSinceYear']=store_raw_df['CompetitionOpenSinceYear'].astype(np.int64)

for index, row in store_raw_df.iterrows():
    store_raw_df.loc[index,'CompetitionSinceDate']=(pd.to_datetime(str(row['CompetitionOpenSinceYear'])+'-'+str(row['CompetitionOpenSinceMonth'])+'-'+str(1)))

count of CompetitionOpenSinceYear!=CompetitionOpenSinceMonth is 0


In [59]:
print 'CompetitionDistance，CompetitionOpenSinceMonth，CompetitionOpenSinceYear同时为缺省值时替换后的结果：'
store_raw_df[store_raw_df['InCompetition']==0]

CompetitionDistance，CompetitionOpenSinceMonth，CompetitionOpenSinceYear同时为缺省值时替换后的结果：


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,InCompetition,CompetitionSinceDate
290,291,d,a,2325.0,1,2016,0,NaN,NaN,NaN,False,2016-01-01
621,622,a,c,2325.0,1,2016,0,NaN,NaN,NaN,False,2016-01-01
878,879,d,a,2325.0,1,2016,1,5.0,2013.0,"Feb,May,Aug,Nov",False,2016-01-01


In [60]:


print 'CompetitionDistance不缺省，而CompetitionOpenSinceMonth，CompetitionOpenSinceYear为缺省值时替换后的结果：'
store_raw_df[store_raw_df['CompetitionOpenSinceYear']==1961].head()

CompetitionDistance不缺省，而CompetitionOpenSinceMonth，CompetitionOpenSinceYear为缺省值时替换后的结果：


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,InCompetition,CompetitionSinceDate
11,12,a,c,1070.0,1,1961,1,13.0,2010.0,"Jan,Apr,Jul,Oct",True,1961-01-01
12,13,d,a,310.0,1,1961,1,45.0,2009.0,"Feb,May,Aug,Nov",True,1961-01-01
15,16,a,c,3270.0,1,1961,0,NaN,NaN,NaN,True,1961-01-01
18,19,a,c,3240.0,1,1961,1,22.0,2011.0,"Mar,Jun,Sept,Dec",True,1961-01-01
21,22,a,a,1040.0,1,1961,1,22.0,2012.0,"Jan,Apr,Jul,Oct",True,1961-01-01


#### 1.3.2 store_raw_df的Promo数据处理
- 目的：
    - store_raw_df中添加特征Promo2SinceDate
    - PromoInterval 将缺失值替代为''
- 如果Promo2为0，那么Promo2SinceDate为test样本之后的日期(本项目取为2016-W1-Monday)

In [61]:

# 将所有为Promo2为0的Promo2SinceWeek/Year设置为2016 Week-01
store_raw_df['Promo2SinceWeek'].fillna(1,inplace=True)
store_raw_df['Promo2SinceYear'].fillna(2016,inplace=True)
# 将Promo2SinceWeek/Year格式改为np.int64
store_raw_df['Promo2SinceWeek']=store_raw_df['Promo2SinceWeek'].astype(np.int64)
store_raw_df['Promo2SinceYear']=store_raw_df['Promo2SinceYear'].astype(np.int64)

for index, row in store_raw_df.iterrows():
    store_raw_df.loc[index,'Promo2SinceDate']=Week(row['Promo2SinceYear'], row['Promo2SinceWeek'] ).monday()

store_raw_df['PromoInterval'].fillna('',inplace=True)

#### 1.3.3 处理store_raw_df的StoreType和Assortment
- 转换为分类特征

In [62]:
store_raw_df['StoreType_cat'] = store_raw_df['StoreType'].astype('category').cat.codes.astype(np.int64)
store_raw_df['Assortment_cat'] = store_raw_df['Assortment'].astype('category').cat.codes.astype(np.int64)

### 1.4 将train、est和store数据进行合并，并进行处理
#### 1.4.1 将train/test和store数据进行合并

In [63]:

train_store_raw_df=train_raw_df.merge(store_raw_df,on='Store')
test_store_raw_df=test_raw_df.merge(store_raw_df,on='Store')

#### 1.4.2 train_store_raw_df,test_store_raw_df中的Competition数据处理
- 目的：
    - train_store_raw_df中添加特征InCompetitionToday,DaysCountSinceCompetition
    - 注意：在计算好的DaysCountSinceCompetition后加1，这样如果已经开始Competition的药店最小值为1，而DaysCountSinceCompetition为0时仅表示那些还没有开始竞争的药店；

In [65]:

#train_store_raw_df中添加特征InCompetitionToday,DaysCountSinceCompetition
train_store_raw_df['InCompetitionToday']=pd.to_datetime(train_store_raw_df['CompetitionSinceDate'])<(train_store_raw_df['Date_time'])
train_store_raw_df['DaysCountSinceCompetition']=0

mask=train_store_raw_df['InCompetitionToday']==True
train_store_raw_df.loc[mask,'DaysCountSinceCompetition']=(((train_store_raw_df.loc[mask,'Date_time'])-pd.to_datetime(train_store_raw_df.loc[mask,'CompetitionSinceDate'])) / np.timedelta64(1, 'D')).astype(int)+1.0

#test_store_raw_df中添加特征InCompetitionToday,DaysCountSinceCompetition
test_store_raw_df['InCompetitionToday']=pd.to_datetime(test_store_raw_df['CompetitionSinceDate'])<(test_store_raw_df['Date_time'])
test_store_raw_df['DaysCountSinceCompetition']=0

mask=test_store_raw_df['InCompetitionToday']==True
test_store_raw_df.loc[mask,'DaysCountSinceCompetition']=(((test_store_raw_df.loc[mask,'Date_time'])-pd.to_datetime(test_store_raw_df.loc[mask,'CompetitionSinceDate'])) / np.timedelta64(1, 'D')).astype(int)+1.0

#### 1.4.3 train_store_raw_df中的Promo2数据处理
- 目的：
    - train_store_raw_df中添加特征InPromo2Today,DaysCountSincePromo2
    - 注意：在计算好的天数后加1，这样如果已经开始Promo2的药店最小值为1，而DaysCountSincePromo2为0时仅表示那些还没有开始促销的药店；

In [66]:

train_store_raw_df['InPromo2Today']=0
mask=(pd.to_datetime(train_store_raw_df['Promo2SinceDate'])<(train_store_raw_df['Date_time']))&(train_store_raw_df['PromoInterval']!='')
for interval in train_store_raw_df[mask].PromoInterval.unique():
    for month in interval.split(','):
        train_store_raw_df.loc[mask & (train_store_raw_df['Month-Str']==month)&(train_store_raw_df['PromoInterval']==interval) ,
                              'InPromo2Today']=1        
        
train_store_raw_df['DaysCountSincePromo2']=0
mask=train_store_raw_df['InPromo2Today']==True
train_store_raw_df.loc[mask,'DaysCountSincePromo2']=(((train_store_raw_df.loc[mask,'Date_time'])-pd.to_datetime(train_store_raw_df.loc[mask,'Promo2SinceDate'])) / np.timedelta64(1, 'D')).astype(int)+1.0

test_store_raw_df['InPromo2Today']=0
mask=(pd.to_datetime(test_store_raw_df['Promo2SinceDate'])<(test_store_raw_df['Date_time']))&(test_store_raw_df['PromoInterval']!='')
for interval in test_store_raw_df[mask].PromoInterval.unique():
    for month in interval.split(','):
        test_store_raw_df.loc[mask & (test_store_raw_df['Month-Str']==month)&(test_store_raw_df['PromoInterval']==interval) ,
                              'InPromo2Today']=1        
        
test_store_raw_df['DaysCountSincePromo2']=0
mask=test_store_raw_df['InPromo2Today']==True
test_store_raw_df.loc[mask,'DaysCountSincePromo2']=(((test_store_raw_df.loc[mask,'Date_time'])-pd.to_datetime(test_store_raw_df.loc[mask,'Promo2SinceDate'])) / np.timedelta64(1, 'D')).astype(int)+1.0



### 1.5 数据范围调整
- 将连续量离散，方便进行embedding操作
- 调整参数
    - CompetitionDistance -->log2
    - DaysCountSincePromo2 --> log2
    - DaysCountSinceCompetition ---> log2

In [67]:

test_store_raw_df['CompetitionDistance_log']=np.round(np.log2(test_store_raw_df['CompetitionDistance']+1.0))
test_store_raw_df['DaysCountSinceCompetition_log']=np.round(np.log2(test_store_raw_df['DaysCountSinceCompetition']+1.0))
test_store_raw_df['DaysCountSincePromo2_log']=np.round(np.log2(test_store_raw_df['DaysCountSincePromo2']+1.0))
train_store_raw_df['CompetitionDistance_log']=np.round(np.log2(train_store_raw_df['CompetitionDistance']+1.0))
train_store_raw_df['DaysCountSinceCompetition_log']=np.round(np.log2(train_store_raw_df['DaysCountSinceCompetition']+1.0))
train_store_raw_df['DaysCountSincePromo2_log']=np.round(np.log2(train_store_raw_df['DaysCountSincePromo2']+1.0))

In [68]:
train_store_raw_df['CompetitionDistance_log'].unique()

array([ 10.,   9.,  14.,  15.,   8.,  13.,  11.,  12.,   6.,   5.,   7.,
        16.,   4.])

### 1.6 预览数据

In [69]:
### 对数据进行预览 1
print('--------train.csv---------')
display(train_raw_df.head(n=2))
print('--------test.csv---------')
display(test_raw_df.head(n=2))
print('--------store.csv---------')
display(store_raw_df.head(n=2))
print('--------train_store_raw_df---------')
display(train_store_raw_df.head(n=2))
print('--------test_store_raw_df---------')
display(test_store_raw_df.head(n=2))

--------train.csv---------


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Date_time,Year,Month,Day,Year-Month,Month-Str,DayOfYear,SalesLog,StateHoliday_cat,Outlier_3
0,1,5,2015-07-31,5263,555,1,1,0,1,2015-07-31,2015,7,31,2015-07,Jul,212,8.568456,0,False
1,2,5,2015-07-31,6064,625,1,1,0,1,2015-07-31,2015,7,31,2015-07,Jul,212,8.710125,0,False


--------test.csv---------


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Date_time,Year,Month,Day,Year-Month,Month-Str,DayOfYear,StateHoliday_cat
0,1,1,4,2015-09-17,1.0,1,0,0,2015-09-17,2015,9,17,2015-09,Sept,260,0
1,2,3,4,2015-09-17,1.0,1,0,0,2015-09-17,2015,9,17,2015-09,Sept,260,0


--------store.csv---------


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,InCompetition,CompetitionSinceDate,Promo2SinceDate,StoreType_cat,Assortment_cat
0,1,c,a,1270.0,9,2008,0,1,2016,,True,2008-09-01,2016-01-04,2,0
1,2,a,a,570.0,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",True,2007-11-01,2010-03-29,0,0


--------train_store_raw_df---------


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Date_time,...,Promo2SinceDate,StoreType_cat,Assortment_cat,InCompetitionToday,DaysCountSinceCompetition,InPromo2Today,DaysCountSincePromo2,CompetitionDistance_log,DaysCountSinceCompetition_log,DaysCountSincePromo2_log
0,1,5,2015-07-31,5263,555,1,1,0,1,2015-07-31,...,2016-01-04,2,0,True,2525.0,0,0.0,10.0,11.0,0.0
1,1,4,2015-07-30,5020,546,1,1,0,1,2015-07-30,...,2016-01-04,2,0,True,2524.0,0,0.0,10.0,11.0,0.0


--------test_store_raw_df---------


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Date_time,Year,...,Promo2SinceDate,StoreType_cat,Assortment_cat,InCompetitionToday,DaysCountSinceCompetition,InPromo2Today,DaysCountSincePromo2,CompetitionDistance_log,DaysCountSinceCompetition_log,DaysCountSincePromo2_log
0,1,1,4,2015-09-17,1.0,1,0,0,2015-09-17,2015,...,2016-01-04,2,0,True,2573.0,0,0.0,10.0,11.0,0.0
1,857,1,3,2015-09-16,1.0,1,0,0,2015-09-16,2015,...,2016-01-04,2,0,True,2572.0,0,0.0,10.0,11.0,0.0


In [70]:
#对数据进行预览 2
print('--------train.csv---------')
train_raw_df.info()
print('--------test.csv---------')
test_raw_df.info()
print('--------store.csv---------')
store_raw_df.info()
print('--------train_store_raw_df---------')
train_store_raw_df.info()
print('--------test_store_raw_df---------')
test_store_raw_df.info()

--------train.csv---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
Store               1017209 non-null int64
DayOfWeek           1017209 non-null int64
Date                1017209 non-null object
Sales               1017209 non-null int64
Customers           1017209 non-null int64
Open                1017209 non-null int64
Promo               1017209 non-null int64
StateHoliday        1017209 non-null object
SchoolHoliday       1017209 non-null int64
Date_time           1017209 non-null datetime64[ns]
Year                1017209 non-null int64
Month               1017209 non-null int64
Day                 1017209 non-null int64
Year-Month          1017209 non-null object
Month-Str           1017209 non-null object
DayOfYear           1017209 non-null int64
SalesLog            1017209 non-null float64
StateHoliday_cat    1017209 non-null int64
Outlier_3           844392 non-null object
dtypes: datetime64[ns](1), f

### 1.6 将处理好的数据保存本地硬盘
- feature_x_list & feature_y_list
- train_store_raw_df & test_store_raw_df

In [71]:
#feature
feature_x_list=['Store','DayOfWeek',#'Open',
                'Year','Month','Day','DayOfYear',
                
                'StoreType_cat','Assortment_cat','StateHoliday_cat','SchoolHoliday',
                
                'Promo','Promo2','InPromo2Today','DaysCountSinceCompetition_log',

#                 'Outlier_3','Outlier_2','Outlier_2_5',
                'InCompetition',
                'InCompetitionToday','CompetitionDistance_log','DaysCountSincePromo2_log'                
               ]
feature_y_list=['Sales']
#write file
file_train_store_raw_df='train_store_raw_df.pickle'
file_test_store_raw_df='test_store_raw_df.pickle'
file_feature='feature_x_list.pickle'
path='data_inter_transfer'

train_store_raw_df.to_pickle(os.path.join(path, file_train_store_raw_df))
test_store_raw_df.to_pickle(os.path.join(path, file_test_store_raw_df))

(pd.Series(feature_x_list)).to_pickle(os.path.join(path, file_feature))

file_train_raw_df='train_raw_df.pickle'
train_raw_df.to_pickle(os.path.join(path, file_train_raw_df))

file_store_raw_df='store_raw_df.pickle'
store_raw_df.to_pickle(os.path.join(path, file_store_raw_df))
